## Libraries and helper  functions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type).startswith('int'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int64)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float64)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print('Mem. usage decreased from {:5.2f} Mb to {:5.2f} Mb ({:.1f}% reduction)'.format(
            start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem
        ))
    return df

## Import data

In [3]:
users = pd.read_csv("user_contract_id.csv", sep=";")
users.head(5)

,Number of Records,contract_id,user
0,1,f58dbbbe67142f3b6023965c7b9cc120,d13e08d3a4c660ab22ea011efe7285e7
1,1,30433a23c7e86d7e1de340d2c33f9a43,7045239ada9ce128b97e2ad8784560f5
2,1,001a8583feb1edfe7adcebddd55d90b7,1c39b5a73ef576c031b54d94ea6368e4
3,1,30433a23c7e86d7e1de340d2c33f9a43,b9e028301bd78ffd3f6a9df5eae869ab
4,1,f58dbbbe67142f3b6023965c7b9cc120,8ef669a72d2e7a356141381b83ef20a0


In [4]:
contracts = pd.read_csv("contracts_data.csv", sep=";")
contracts.head(5)

,Number of Records,contract_group,contract_id,contract_type
0,1,4c87bee0a400e8da705eb82b601b8683,8836e9016d1077871a32cad80088cca4,787df3c929e166f3f27c5208e45c23d3
1,1,22c1b9370cdf0901047d3588b1c60eb0,2f57666897fc2c6c5b4728bfebe2311f,cbdaba5605992f8e1635ad782152d65a
2,1,22c1b9370cdf0901047d3588b1c60eb0,579025104268a96dca7d9e5544b7f4df,cbdaba5605992f8e1635ad782152d65a
3,1,4c87bee0a400e8da705eb82b601b8683,609bc1c1cc44a4b7bd43d3ae90061437,787df3c929e166f3f27c5208e45c23d3
4,1,4c87bee0a400e8da705eb82b601b8683,49f54d6ff987b0bff16ea532097ea611,787df3c929e166f3f27c5208e45c23d3


In [5]:
banner_data = pd.read_csv("DS_test_Revenue_estimate.csv", sep=";")
banner_data.head(5)

/home/anna/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Number of Records,avg_cost_per_reg,cid_tds,geo,group_user,impression,ispay,log_id,payout,project_id,ts,user,zoneid
0,1,"38,76314",1538631046687qbbcmpvf,US,8-30day,8,no,7f94ea1342f65023fa20290c1e641c5f,NaN,2.0,1538631046,d92631fa78a80606dee1ecf22ff7d51f,180
1,1,"38,76314",1538135942855masphqkt,US,8-30day,5,no,e6467d039b4e0ac99f7463eb5da2a612,NaN,2.0,1538135942,23f5c8d16b383f932e3ad73546b45166,7
2,1,"38,76314",1537798087995wogywfsp,US,5day,4,no,e6467d039b4e0ac99f7463eb5da2a612,NaN,2.0,1537798087,23f5c8d16b383f932e3ad73546b45166,8
3,1,"38,76314",1538328459891ssrypjnx,US,0day,1,no,e6467d039b4e0ac99f7463eb5da2a612,NaN,2.0,1538328459,122b1d81bcb352d38a5ed29132e37f5d,215
4,1,"38,76314",1537656220532wamztscd,GB,0day,1,no,e6467d039b4e0ac99f7463eb5da2a612,NaN,2.0,1537656220,50d8db7813038ca01b6d2c311988d679,11


## Preprocessing

In [6]:
users.shape

(605907, 3)

In [7]:
users = users.drop(['Number of Records'], axis=1)
users['user'] = users['user'].apply(int, base=16)
users['user'] = pd.to_numeric(users['user'], errors='coerce')
users.shape

(605907, 2)

In [8]:
contracts = contracts.drop(['Number of Records'], axis=1)

In [9]:
banner_data['user'] = banner_data['user'].apply(int, base=16)
banner_data['user'] = pd.to_numeric(banner_data['user'], errors='coerce')
banner_data.shape

(32918966, 13)

In [10]:
payout_groups = banner_data[['payout','cid_tds']].groupby('payout').count()
payout_groups

,cid_tds
payout,
0,5036
"0,3652",84
"0,5478",30
"0,7304",5
"0,913",6
...,...
"95,865",3
"97,691",2
"97,8736",2


In [11]:
banner_data['payout_num'] = pd.to_numeric(banner_data['payout'].str.replace(',','.'))

In [12]:
payout_groups_2 = banner_data[['payout_num','cid_tds']].groupby('payout_num').count()
payout_groups_2

,cid_tds
payout_num,
0.000000,5036
0.365200,84
0.547800,30
0.730400,5
0.913000,6
...,...
1462.344796,2
1552.100000,13
1826.000000,13


In [13]:
banner_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32918966 entries, 0 to 32918965
Data columns (total 14 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Number of Records  int64  
 1   avg_cost_per_reg   object 
 2   cid_tds            object 
 3   geo                object 
 4   group_user         object 
 5   impression         int64  
 6   ispay              object 
 7   log_id             object 
 8   payout             object 
 9   project_id         float64
 10  ts                 int64  
 11  user               float64
 12  zoneid             int64  
 13  payout_num         float64
dtypes: float64(3), int64(4), object(7)
memory usage: 3.4+ GB


In [14]:
banner_data['ispay_num'] = np.where(banner_data['ispay']=='is',1,0)

Let's take a look at number of unique values in variables which are candidates to one-hot encoding.

'Geo' variable contains too much unique values for one-hot encoding. Other variables can be one-hot encoded.

'Payout' (our dependent variable) also contains too many values for the multi-class classification, so we have regression problem. 

In [15]:
banner_data = banner_data.drop(['Number of Records','log_id','avg_cost_per_reg','ispay','geo','payout'], axis=1)

In [16]:
reduce_mem_usage(banner_data)

Mem. usage decreased from 2260.37 Mb to 1695.27 Mb (25.0% reduction)


,cid_tds,group_user,impression,project_id,ts,user,zoneid,payout_num,ispay_num
0,1538631046687qbbcmpvf,8-30day,8,2.0,1538631046,2.886408e+38,180,NaN,0
1,1538135942855masphqkt,8-30day,5,2.0,1538135942,4.779917e+37,7,NaN,0
2,1537798087995wogywfsp,5day,4,2.0,1537798087,4.779917e+37,8,NaN,0
3,1538328459891ssrypjnx,0day,1,2.0,1538328459,2.414997e+37,215,NaN,0
4,1537656220532wamztscd,0day,1,2.0,1537656220,1.074642e+38,11,NaN,0
...,...,...,...,...,...,...,...,...,...
32918961,1538197982122mlxrvjgv,8-30day,447,2.0,1538197982,1.839832e+38,215,NaN,0
32918962,1538198085129gydqzlsq,8-30day,452,2.0,1538198085,1.839832e+38,215,NaN,0
32918963,1538222182321oxgdazoa,8-30day,719,2.0,1538222182,1.839832e+38,215,NaN,0
32918964,1538221516473vucwhxwx,8-30day,654,2.0,1538221516,1.839832e+38,1,NaN,0


In [17]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605907 entries, 0 to 605906
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   contract_id  605881 non-null  object 
 1   user         605907 non-null  float64
dtypes: float64(1), object(1)
memory usage: 9.2+ MB


In [18]:
contracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967 entries, 0 to 966
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   contract_group  959 non-null    object
 1   contract_id     959 non-null    object
 2   contract_type   959 non-null    object
dtypes: object(3)
memory usage: 22.8+ KB


In [19]:
banner_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32918966 entries, 0 to 32918965
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   cid_tds     object 
 1   group_user  object 
 2   impression  int64  
 3   project_id  float32
 4   ts          int64  
 5   user        float64
 6   zoneid      int32  
 7   payout_num  float32
 8   ispay_num   int16  
dtypes: float32(2), float64(1), int16(1), int32(1), int64(2), object(2)
memory usage: 1.7+ GB


### Clean data

Drop rows in table "Contracts" with missing contract_id, since they do not give any information to us.

In [20]:
contracts = contracts.dropna()
contracts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 959 entries, 0 to 964
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   contract_group  959 non-null    object
 1   contract_id     959 non-null    object
 2   contract_type   959 non-null    object
dtypes: object(3)
memory usage: 30.0+ KB


In [21]:
users.shape

(605907, 2)

In [22]:
users.drop_duplicates(ignore_index=True, inplace=True, subset=['user'], keep='last')
users.shape

(605896, 2)

In [23]:
contracts.drop_duplicates(ignore_index=True, inplace=True, subset=['contract_id', 'contract_type'], keep='last')
contracts.shape

(959, 3)

In [24]:
contracts.nunique()

contract_group    126
contract_id       959
contract_type       9
dtype: int64

Contract_type variable can be one-hot encoded. Let's add it to dataset.

In [25]:
users_in_contracts = contracts.merge(users, how='inner', on='contract_id')
users_in_contracts = users_in_contracts.drop(['contract_id','contract_group'], axis=1)
users_in_contracts.shape

(605870, 2)

In [26]:
banner_data = banner_data.merge(users_in_contracts, how='left', on='user')

## Exploratory data analysis and feature design

In [27]:
group_user_groups = banner_data[['group_user','cid_tds']].groupby('group_user').count()
group_user_groups

,cid_tds
group_user,
0day,8841598
181-360day,3159251
1day,2130529
2day,1218841
31-60day,2732591
361day,1135308
3day,903881
4day,740441
5day,618601


In [28]:
impression_groups = banner_data[['impression','cid_tds']].groupby('impression').count()
impression_groups

,cid_tds
impression,
1,4690822
2,1562725
3,931543
4,628414
5,495475
...,...
83943,1
83944,1
83945,2


In [29]:
banner_data = banner_data.assign(impression_group=pd.cut(banner_data['impression'], 
                               bins=[0, 1, 2, 3, 4, 5, 10, 20, 50, 100, 100000], 
                               labels=['1', '2', '3', '4', '5', '6-10','11-20', '21-50', '50-100','>100']))

In [30]:
impression_groups = banner_data[['impression_group','cid_tds']].groupby('impression_group').count()
impression_groups

,cid_tds
impression_group,
1,4690822
2,1562725
3,931543
4,628414
5,495475
6-10,1746485
11-20,2069222
21-50,3327371
50-100,3062140


In [31]:
banner_data = banner_data.drop(['impression'], axis=1)

In [32]:
project_groups = banner_data[['project_id','cid_tds']].groupby('project_id').count()
project_groups

,cid_tds
project_id,
1.0,3774003
2.0,29144877


In [33]:
zoneid_groups = banner_data[['zoneid','cid_tds']].groupby('zoneid').count()
zoneid_groups

,cid_tds
zoneid,
1,7258975
4,1651925
5,672180
6,668587
7,220357
8,706199
9,165404
10,2579424
11,2658674


In [34]:
banner_data['hour2'] = pd.to_datetime(banner_data['ts'], unit='s').dt.hour

In [35]:
banner_data = banner_data.assign(hour_group=pd.cut(banner_data['hour2'], 
                               bins=[0, 4, 8, 12, 16, 20, 25], 
                               labels=['0-4', '5-8', '9-12', '13-16', '17-20','21-24']))

In [36]:
banner_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32918966 entries, 0 to 32918965
Data columns (total 12 columns):
 #   Column            Dtype   
---  ------            -----   
 0   cid_tds           object  
 1   group_user        object  
 2   project_id        float32 
 3   ts                int64   
 4   user              float64 
 5   zoneid            int32   
 6   payout_num        float32 
 7   ispay_num         int16   
 8   contract_type     object  
 9   impression_group  category
 10  hour2             int64   
 11  hour_group        category
dtypes: category(2), float32(2), float64(1), int16(1), int32(1), int64(2), object(3)
memory usage: 2.2+ GB


In [37]:
banner_data = banner_data.drop(['user'], axis=1)

In [38]:
banner_data['project_id'] = banner_data['project_id'].astype('category')

In [39]:
banner_data['zoneid'] = banner_data['zoneid'].astype('category')

In [40]:
banner_data['contract_type'] = banner_data['contract_type'].astype('category')

In [41]:
banner_data['group_user'] = banner_data['group_user'].astype('category')

In [42]:
banner_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32918966 entries, 0 to 32918965
Data columns (total 11 columns):
 #   Column            Dtype   
---  ------            -----   
 0   cid_tds           object  
 1   group_user        category
 2   project_id        category
 3   ts                int64   
 4   zoneid            category
 5   payout_num        float32 
 6   ispay_num         int16   
 7   contract_type     category
 8   impression_group  category
 9   hour2             int64   
 10  hour_group        category
dtypes: category(6), float32(1), int16(1), int64(2), object(1)
memory usage: 1.3+ GB


One-hot encoding of "zoneid" variable results in error due to low memory. Let's one-hot encode only 'group_user','project_id','contract_type','impression_group', 'hour' variables.

In [43]:
train_val_test = pd.get_dummies(banner_data, columns=['group_user','project_id','contract_type','impression_group'])

In [44]:
train_val_test['payout_num'] = train_val_test['payout_num'].fillna(0)

Convert numeric timestamp to datetime.

In [45]:
train_val_test['ts'] = pd.to_datetime(train_val_test['ts'], unit='s')
train_val_test['ts'].describe(datetime_is_numeric=True)

count                         32918966
mean     2018-09-28 19:41:12.816581376
min                2018-09-19 21:00:00
25%                2018-09-24 04:15:29
50%                2018-09-28 19:22:27
75%                2018-10-03 10:13:10
max                2018-10-07 20:59:59
Name: ts, dtype: object

According to description, we have records for interval September 20th - October 7th. Thus, we need to drop records with dates not in this interval.

## Modelling

Extract test dataset

In [46]:
test = train_val_test[train_val_test['ts'] >= '2018-10-04 00:00:00']

In [47]:
test = pd.get_dummies(test, columns=['hour_group'])

Get random sample for model training

In [48]:
train_val = train_val_test.sample(n = 1500000)

In [49]:
train_val = train_val[(train_val['ts'] < '2018-10-04 00:00:00') & (train_val['ts'] >= '2018-09-20 00:00:00')]
train_val.shape

(1161857, 42)

In [50]:
train_val = pd.get_dummies(train_val, columns=['hour_group'])

Drop excess columns

In [51]:
X = train_val.drop(['cid_tds','ts','zoneid','payout_num','hour2'], axis=1)
y = train_val['payout_num']

Split dataset into train / validation 

In [52]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

Train models

#### Linear regression - baseline model

In [53]:
lm = LinearRegression()
lm.fit(X_train, y_train)

Yhat = lm.predict(X_val) 

In [54]:
print('The R-square is: ', r2_score(y_val, Yhat))

The R-square is:  0.0002260389986707967


In [55]:
print('The Mean Square Error on validation dataset is: ', mean_squared_error(y_val, Yhat))

The Mean Square Error on validation dataset is:  2.523806364325739


#### Random forest regression

In [59]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, y_train)

Yhat = regressor.predict(X_val) 

In [60]:
print('The R-square is: ', r2_score(y_val, Yhat))

The R-square is:  -0.01965426985601537


In [61]:
print('The Mean Square Error on validation dataset is: ', mean_squared_error(y_val, Yhat))

The Mean Square Error on validation dataset is:  2.6837319747038184


#### XGBoost regression

In [56]:
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

Yhat = model.predict(X_val) 

In [57]:
print('The R-square is: ', r2_score(y_val, Yhat))

The R-square is:  -0.011407352393239645


In [58]:
print('The Mean Square Error on validation dataset is: ', mean_squared_error(y_val, Yhat))

The Mean Square Error on validation dataset is:  2.6620257


## Prediction

In [54]:
x_test = test.drop(['cid_tds','ts','zoneid','payout_num','hour2'], axis=1)

In [55]:
x_test.columns.tolist()

['ispay_num',
 'group_user_0day',
 'group_user_181-360day',
 'group_user_1day',
 'group_user_2day',
 'group_user_31-60day',
 'group_user_361day',
 'group_user_3day',
 'group_user_4day',
 'group_user_5day',
 'group_user_61-90day',
 'group_user_6day',
 'group_user_7day',
 'group_user_8-30day',
 'group_user_91-180day',
 'project_id_1.0',
 'project_id_2.0',
 'contract_type_0cbc6611f5540bd0809a388dc95a615b',
 'contract_type_787df3c929e166f3f27c5208e45c23d3',
 'contract_type_7a1920d61156abc05a60135aefe8bc67',
 'contract_type_a8ec3492f1a1296c01d725428c42bb2c',
 'contract_type_ae5b813b9b18f8760d5a612341a09f67',
 'contract_type_cbdaba5605992f8e1635ad782152d65a',
 'contract_type_db407837f9f3103c702b5e6184f26c5d',
 'contract_type_dc5b2c63368e976b65ab1f2493b7c3be',
 'contract_type_e854a6217ff1e49413ffa5353f124c64',
 'impression_group_1',
 'impression_group_2',
 'impression_group_3',
 'impression_group_4',
 'impression_group_5',
 'impression_group_6-10',
 'impression_group_11-20',
 'impression_grou

In [56]:
X_val.columns.tolist()

['ispay_num',
 'group_user_0day',
 'group_user_181-360day',
 'group_user_1day',
 'group_user_2day',
 'group_user_31-60day',
 'group_user_361day',
 'group_user_3day',
 'group_user_4day',
 'group_user_5day',
 'group_user_61-90day',
 'group_user_6day',
 'group_user_7day',
 'group_user_8-30day',
 'group_user_91-180day',
 'project_id_1.0',
 'project_id_2.0',
 'contract_type_0cbc6611f5540bd0809a388dc95a615b',
 'contract_type_787df3c929e166f3f27c5208e45c23d3',
 'contract_type_7a1920d61156abc05a60135aefe8bc67',
 'contract_type_a8ec3492f1a1296c01d725428c42bb2c',
 'contract_type_ae5b813b9b18f8760d5a612341a09f67',
 'contract_type_cbdaba5605992f8e1635ad782152d65a',
 'contract_type_db407837f9f3103c702b5e6184f26c5d',
 'contract_type_dc5b2c63368e976b65ab1f2493b7c3be',
 'contract_type_e854a6217ff1e49413ffa5353f124c64',
 'impression_group_1',
 'impression_group_2',
 'impression_group_3',
 'impression_group_4',
 'impression_group_5',
 'impression_group_6-10',
 'impression_group_11-20',
 'impression_grou

In [57]:
x_test.shape

(7133130, 42)

In [58]:
part1 = x_test.head(4000000)
part2 = x_test.tail(x_test.shape[0]-part1.shape[0])

In [59]:
part2.shape

(3133130, 42)

In [60]:
y_test1 = lm.predict(part1)

In [61]:
y_test2 = lm.predict(part2)

In [62]:
y_test = pd.concat([pd.Series(y_test1),pd.Series(y_test2)])

In [64]:
df = pd.DataFrame(y_test, columns=['prediction'])

In [65]:
df.to_csv('prediction.csv', index=False)

In [66]:
cid_test = test['cid_tds']

In [68]:
df2 = pd.DataFrame(cid_test, columns=['cid_tds'])

In [69]:
df2.to_csv('cid_tds_test.csv', index=False)